In [ ]:
import pandas as pd
import numpy as np
import os
import re

# 헤드라인 뉴스 csv

In [ ]:
# CSV 파일이 들어있는 폴더 경로
csv_folder_path = "../dataset/navernews"

# 폴더 내에 있는 모든 파일 목록 가져오기
file_list = os.listdir(csv_folder_path)

# CSV 파일만 선택하여 읽어서 리스트에 저장
csv_data_list = []
for file_name in file_list:
    if file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder_path, file_name)
        csv_data = pd.read_csv(file_path)
        csv_data_list.append(csv_data)

- 중복되는 행 제거는 news_crawler에 포함되어 있음

In [ ]:
# 중복되는 행 제거
csv_cleaned = []
for csv in csv_data_list:
    csv_cleaned.append(csv.drop_duplicates())

# 뉴스 본문 및 요약문 csv

- Label을 달기 위해 전처리

In [ ]:
# CSV 파일이 들어있는 폴더 경로
csv_folder_path = "../dataset/navernews/text"

# 폴더 내에 있는 모든 파일 목록 가져오기
file_list = os.listdir(csv_folder_path)

# CSV 파일만 선택하여 읽어서 리스트에 저장
csv_data_list = []
for file_name in file_list:
    if file_name.endswith(".csv"):
        file_path = os.path.join(csv_folder_path, file_name)
        csv_data = pd.read_csv(file_path)
        csv_data_list.append(csv_data)

In [ ]:
def excluding_text(modified_text):
    pattern1 = re.compile(r'\s\([^)]*\)')
    modified_text = re.sub(pattern1, ' ', modified_text)
    pattern2 = re.compile(r'\([^)]*\)\s')
    modified_text = re.sub(pattern2, ' ', modified_text)
    pattern3 = re.compile(r'\([^)]*\)') 
    modified_text = re.sub(pattern3, '', modified_text)
    pattern4 = re.compile(r'\[[^\]]*\]')
    modified_text = re.sub(pattern4, '', modified_text)
    return modified_text

In [ ]:
def preprocessing_text(text):
    modified_text = text.replace(". ", ".\n")
    modified_text = excluding_text(modified_text)
    modified_text = modified_text.split("\n")
    cleaned_sentences = []
    for sentence in modified_text:
        modified_sentence = re.sub(r'\s\s+', " ", sentence)
        modified_sentence = re.sub(r'“([^“”]+)”', r'"\1"', modified_sentence)
        modified_sentence = re.sub(r'‘([^‘’]+)’', r"'\1'", modified_sentence)
        modified_sentence = re.sub(r'[^\w\s.?!]', '', modified_sentence)
        if modified_sentence.strip():
            cleaned_sentences.append(modified_sentence.strip())
    return cleaned_sentences

def preprocessing_summary(text):
    modified_texts = text.split("\n")
    modified_texts = modified_texts[1:]
    modified_text = []
    for i, t in enumerate(modified_texts):
        modified_texts[i] = t.replace(". ", ".\n")
    for i, t in enumerate(modified_texts):
        for ts in t.split("\n"):
            modified_text.append(ts)
    modified_text = list(map(excluding_text, modified_text))
    cleaned_sentences = []
    for sentence in modified_text:
        modified_sentence = re.sub(r'“([^“”]+)”', r'"\1"', sentence)
        modified_sentence = re.sub(r'‘([^‘’]+)’', r"'\1'", modified_sentence)
        modified_sentence = re.sub(r'[^\w\s.?!]', '', modified_sentence)
        if modified_sentence.strip():
            cleaned_sentences.append(modified_sentence)
    return cleaned_sentences

def add_label(row):
    label = []
    for idx, b in enumerate(row['body']):
        if b in row['summary']:
            label.append(idx)
    return label


In [ ]:
def test(i,j):
    for id, k in enumerate(preprocessing_text(csv_data_list[i].iloc[j,0])):
        print(id, k)
    print("---")
    for id, m in enumerate(preprocessing_summary(csv_data_list[i].iloc[j,1])):
        print(id, m)
    label = []
    for s, text in enumerate(preprocessing_text(csv_data_list[i].iloc[j,0])):
        if text in preprocessing_summary(csv_data_list[i].iloc[j,1]):
            label.append(s)
    print(label)
    return label

- dataframe 합치기

In [ ]:
new_df = pd.concat([df for df in csv_data_list], axis = 0, ignore_index=True)

- drop na

In [ ]:
preprocessed_df = new_df
print(len(preprocessed_df))
preprocessed_df = preprocessed_df.dropna()
print(len(preprocessed_df))

In [ ]:
preprocessed_df['body'] =  preprocessed_df['body'].apply(lambda x: preprocessing_text(x))
preprocessed_df['summary'] = preprocessed_df['summary'].apply(lambda x: preprocessing_summary(x))
preprocessed_df['label'] = preprocessed_df.apply(add_label, axis=1)

- 1338개 기사 중 1039개가 정확히 label 3개
- 이것만 사용

In [ ]:
preprocessed_df['label'].map(len).value_counts()

In [ ]:
preprocessed_df = preprocessed_df[preprocessed_df['label'].map(len) == 3]

In [ ]:
preprocessed_df.to_csv("../dataset/navernews/train_data.csv", index = False)